In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from openai import AsyncOpenAI, OpenAI
import os
from pathlib import Path
import random
import subprocess
import time

# Start the vllm serve process and redirect stdout/stderr to log files
log_dir = "./logs"
os.makedirs(log_dir, exist_ok=True)
api_key = "sk-" + "".join(
    random.choices(
        "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", k=32
    )
)  # Generate 32-char random key
vllm_serve_command = [
    "vllm",
    "serve",
    str(Path("./models/test/0001").absolute()),
    "--api-key",
    api_key,
    "--served-model-name",
    "default",
]
print(f"Starting vllm serve with command: {' '.join(vllm_serve_command)}")
stdout_log = open(f"{log_dir}/vllm_stdout.log", "w")
stderr_log = open(f"{log_dir}/vllm_stderr.log", "w")
vllm_process = subprocess.Popen(
    vllm_serve_command, stdout=stdout_log, stderr=stderr_log
)

# To shut down the process later, use vllm_process.terminate() or vllm_process.kill()

client = AsyncOpenAI(api_key=api_key, base_url="http://localhost:8000/v1")
sync_client = OpenAI(api_key=api_key, base_url="http://localhost:8000/v1")

start_time = time.time()
timeout = 90.0
failed = True
while failed:
    try:
        sync_client.chat.completions.create(
            model="default",
            messages=[
                {"role": "user", "content": "Hi!"},
            ],
            max_tokens=1,
        )
        failed = False
    except Exception:
        if time.time() - start_time > timeout:
            raise TimeoutError("VLLM server failed to start")

print(f"VLLM server started in {time.time() - start_time:.2f} seconds ✅")

Starting vllm serve with command: vllm serve /home/ubuntu/atreides/experiments/models/test/0001 --api-key sk-vOY3cgCEQKXSDohH8PeDYCbD1R2EFlPC --served-model-name default
VLLM server started in 48.25 seconds ✅


In [ ]:
from lib.clue import Clue, DeductiveSolver

game = Clue(
    num_players=3,
    elements={
        "suspect": Clue.suspects[:3],
        "weapon": Clue.weapons[:3],
        "room": Clue.rooms[:3],
        # "motive": Clue.motives[:6],
        # "time": Clue.get_times("21:00", "03:00", "1h"),
    },
)
game.play(
    deductive_solver=DeductiveSolver(
        # note_cards_in_hand=False,
        # note_responses_to_suggestions=False,
        # note_cards_that_players_do_not_have=False,
        # check_unique_card_placement_constraints=False,
        # check_player_hand_size_constraints=False,
        check_solution_has_one_and_only_one_card_per_element=False,
        check_one_of_constraints=False,
        check_inverse_one_of_constraints=False,
        merge_and_check_disjoint_inverse_one_of_constraints=False,
        exhaustively_test_possible_assignments=False,
    ),
    check_if_deductive_solver_and_cp_solver_grids_match=False,
)
prompt = game.get_prompt()
await client.chat.completions.create(
    model="default",
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=100,
)

In [16]:
vllm_process.terminate()

In [3]:
from lib.clue import Clue, DeductiveSolver

game = Clue(
    num_players=3,
    elements={
        "suspect": Clue.suspects[:3],
        "weapon": Clue.weapons[:3],
        "room": Clue.rooms[:3],
        # "motive": Clue.motives[:6],
        # "time": Clue.get_times("21:00", "03:00", "1h"),
    },
)
game.play(
    deductive_solver=DeductiveSolver(
        # note_cards_in_hand=False,
        # note_responses_to_suggestions=False,
        # note_cards_that_players_do_not_have=False,
        # check_unique_card_placement_constraints=False,
        # check_player_hand_size_constraints=False,
        check_solution_has_one_and_only_one_card_per_element=False,
        check_one_of_constraints=False,
        check_inverse_one_of_constraints=False,
        merge_and_check_disjoint_inverse_one_of_constraints=False,
        exhaustively_test_possible_assignments=False,
    ),
    check_if_deductive_solver_and_cp_solver_grids_match=False,
)
prompt = game.get_prompt()

Player 1's Hand: {'Mrs. White', 'Candlestick'}
Player 2's Hand: {'Dining Room', 'Lounge'}
Player 3's Hand: {'Miss Scarlet', 'Lead Pipe'}
Solution: {'suspect': 'Mr. Green', 'weapon': 'Knife', 'room': 'Hall'}
Player                1  2  3
Element Card                 
Suspect Miss Scarlet  ✗  ✗  ✓
        Mr. Green     ✗  ✗  ✗
        Mrs. White    ✓  ✗  ✗
Weapon  Candlestick   ✓  ✗  ✗
        Knife         ✗  ✗  ✗
        Lead Pipe     ✗  ✗  ✓
Room    Hall          ✗  ✗  ✗
        Lounge        ✗  ✓  ✗
        Dining Room   ✗  ✓  ✗
Player 1's Simple Solver Grid:
Player                1  2  3
Element Card                 
Suspect Miss Scarlet  ✗      
        Mr. Green     ✗      
        Mrs. White    ✓  ✗  ✗
Weapon  Candlestick   ✓  ✗  ✗
        Knife         ✗      
        Lead Pipe     ✗      
Room    Hall          ✗      
        Lounge        ✗      
        Dining Room   ✗      
Player 1's CP-SAT Solver Grid:
Player                1  2  3
Element Card                 
Suspect Mis

In [ ]:
from vllm.sampling_params import SamplingParams

prompt = """
On a warm spring day Summer, Giselle and Connor sat down to play a casual mystery game.

They assembled 3 decks of cards, each for a separate type of information composed of the following:

Suspect:
- Miss Scarlet
- Mr. Green
- Mrs. White

Weapon:
- Candlestick
- Knife
- Lead Pipe

Room:
- Hall
- Lounge
- Dining Room

After randomly (and blindly) choosing one card from each group and placing them in the middle of the table facedown, they shuffled the remaining cards and dealt out the following to each player:

- Summer: 2 cards
- Giselle: 2 cards ('Lounge', 'Miss Scarlet')
- Connor: 2 cards

The game proceeded as follows:

1. On their turn, a player asked about a set of exactly 3 cards, one from each of the game's categories. (Note: Players could ask about any cards, including those in their own hand.)
2. The player directed this question to the other players in clockwise order, starting with the player to their left.
3. If a player had one or more of the asked-about cards, they had to show one of those cards (of their choice) to the asking player privately. The turn then ended, and play passed to the next player.
4. If a player did not have any of the asked-about cards, they said so, and the question passed to the next player in clockwise order.
5. This continued until either:
    a) A player showed a card to the asking player, or
    b) All the queried players had stated they didn't have any of the asked-about cards.
6. After a player's turn ended (either by being shown a card or having all queried players pass), play moved to the next player in clockwise order.

Here is how the game played out:

Summer asked if anyone had 'Mrs. White' or 'Knife' or 'Dining Room':
- Giselle did not have any of the cards
- Connor showed Summer a card

Giselle asked if anyone had 'Mrs. White' or 'Knife' or 'Lounge':
- Connor did not have any of the cards
- Summer did not have any of the cards

Connor asked if anyone had 'Miss Scarlet' or 'Candlestick' or 'Hall':
- Summer did not have any of the cards
- Giselle showed Connor 'Miss Scarlet'

Summer asked if anyone had 'Mr. Green' or 'Knife' or 'Hall':
- Giselle did not have any of the cards
- Connor did not have any of the cards

At this point, Giselle was able to correctly infer the solution and win the game.

What were the facedown cards in the middle of the table?
""".strip()

output = llm.chat([[dict(role="user", content=prompt)]] * 1, sampling_params=SamplingParams(max_tokens=10_000))  # type: ignore